[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jolin-io/KI2022-tutorial-universal-differential-equations/main?filepath=02%20introduction%20to%20deep%20learning%20in%20julia.ipynb)

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.1-darkmode.webp">
</a>

# Introduction to deep learning in <img height="60px" style='height:60px;display:inline;' alt="Julia" src="https://julialang.org/assets/infra/logo.svg">

There are three well-known deep-learning packages for julia, all written in pure julia:
- [Knet.jl](https://github.com/denizyuret/Knet.jl)
- [Flux.jl](https://github.com/FluxML/Flux.jl)
- [Lux.jl](https://github.com/avik-pal/Lux.jl)

We use Lux. It was created as a more functional alternative to Flux.jl in order to address problems with the implicit parameter handling of its interface. One main application of Lux is the combination with Differential Equations, which we will see the next notebook.

Our goal of this notebook is to introduce Lux with the example of fitting a polynomial.

In [ ]:
import Lux
import NNlib, Plots, Random, Statistics, Zygote, Optimization, OptimizationOptimisers, ComponentArrays

rng = Random.default_rng()
Random.seed!(rng, 12345)

# Let's fit a polynomial

👉 Generate 128 datapoints from the polynomial $y = x² - 2x$ and add some noise.

You need `randn`, and you might use `range` for x.

Plot it.

In [ ]:
# your space
# x = ...
# y = ...

## Lux barebones

create a model

In [ ]:
model = Lux.Chain(
    Lux.Dense(1, 16, NNlib.relu),
    Lux.Dense(16, 1),
)

initialize the model

In [ ]:
ps, st = Lux.setup(rng, model)

run the model

In [ ]:
y_pred, st_updated = Lux.apply(model, x, ps, st)

## Training

We use Optimization.jl because it is the one meta package which includes unbelievable many optimization routines, including those typical for deep learning.

Let's define what we want to optimize

In [ ]:
function loss_function(x, y, ps, st)
    y_pred = # 👉 how to get the prediction from our neural network?
    sum(abs2, y .- y_pred), st
end

In [ ]:
loss_function(x, y, ps, st)

Now, you can combine these with your automatic differentiation package of choice.
However, be cautious. Because Lux uses nested NamedTuples for parameters `ps` and states `st`, it might be that your particular autodiff package does not yet support it out-of-the-box.

Or just use Optimization.jl as we do.

In [ ]:
n_iterations = 500

losses = Float64[]
function callback(p, l)
    push!(losses, l)
    if length(losses) % 50 == 0
        Plots.plot(losses, show = :inline, yscale = :log10,
            label = "loss", xlabel = "#epochs", ylabel="loss (log10 scale)")
    end
    # return bool `halt`
    return false
end

# use Ref to handle updates to Lux state `st`
ps_trained, st_trained = let st=Ref(st), x=x, y=y
    
    opt_prob = Optimization.OptimizationProblem(
        Optimization.OptimizationFunction(
            function(ps, constants)
                loss, st[] = loss_function(x, y, ps, st[])
                loss
            end,
            Optimization.AutoZygote()
        ),
        ComponentArrays.ComponentVector(ps),
    )
    
    opt_sol = Optimization.solve(
        opt_prob,
        OptimizationOptimisers.ADAM(0.1),
        callback = callback,
        maxiters = n_iterations,
    )
    
    opt_sol.minimizer, st[]
end

In [ ]:
y_pred, st_pred = Lux.apply(model, x, ps_trained, st_trained)

Plots.plot(x -> evalpoly(x, (0, -2, 1)), x[1, :]; label=false)
Plots.scatter!(x[1, :], y[1, :]; label="Actual Data", markersize=3)
Plots.scatter!(x[1, :], y_pred[1, :]; label="Predictions", markersize=3)

👉 adapt the polynomial and fit something different

In [ ]:
# your space. It is probably easier to just change the above

# That was the introduction to deep learning in julia - Thank you for participating 🙂

Next topic is Universal Differential Equations, i.e. one way to combine deep learning methods with differential equations: [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jolin-io/KI2022-tutorial-universal-differential-equations/main?filepath=03%20deep%20dive%20into%20universal%20differential%20equations.ipynb)

If you have question, suggestions, or you are just interested in Julia, contact me:
- Stephan Sahm stephan.sahm@jolin.io

### Further material
- [Lux.jl documentation](http://lux.csail.mit.edu/stable/)
- especially the [Lux.jl interface](http://lux.csail.mit.edu/stable/manual/interface/)
- and the [NeuralODE example](http://lux.csail.mit.edu/stable/examples/generated/intermediate/NeuralODE/main/)

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.1-darkmode.webp">
</a>

#### Supported by [Jolin.io](https://www.jolin.io)

Jolin.io is an IT-consultancy for high-performance computing and data science

We are there to help you, if you want to
- try out Julia at your company, or
- transition Matlab, Fortran, R, Python, etc. to Julia
- or speed up your existing code